# Pandas 연습문제

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
import warnings
warnings.filterwarnings('ignore') #워닝 무시

## 문제 1(iris)

In [4]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### 1.a
붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width), \
꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등 \
기초통계량(describe())을 구하시오.


In [5]:
iris.groupby(iris.species).agg(['mean', 'std'])

sepal_length           sepal_width           petal_length  \
                   mean       std        mean       std         mean   
species                                                                
setosa            5.006  0.352490       3.428  0.379064        1.462   
versicolor        5.936  0.516171       2.770  0.313798        4.260   
virginica         6.588  0.635880       2.974  0.322497        5.552   

                     petal_width            
                 std        mean       std  
species                                     
setosa      0.173664       0.246  0.105386  
versicolor  0.469911       1.326  0.197753  
virginica   0.551895       2.026  0.274650

## 1.b 
>. 3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다.
즉,
Q1 - 1.5 * (Q3 - Q1) 보다 작은 데이터
Q3 + 1.5 * (Q3 - Q1) 보다 큰 데이터
이 이상치를 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.

In [6]:
s = iris[iris.species=='setosa']['sepal_width']
s.mean(), s.std()

(3.428000000000001, 0.3790643690962886)

In [7]:
q1, q3 = s.quantile(.25), s.quantile(0.75)
iqr = q3 - q1
q1, iqr, q3

(3.2, 0.4750000000000001, 3.6750000000000003)

In [8]:
lower = q1 -1.5 * iqr
upper = q3 +1.5 * iqr
s[(s < lower) | (s > upper)] = np.nan #() 중요하다
s.isnull().sum()

2

In [9]:
s.mean(), s.std() #이상치 제거후 결과

(3.4312500000000004, 0.32034306743094015)

In [10]:
def get_new_stat(s):
    x1, x3 = s.quantile(.25), s.quantile(.75)
    iqr = q3 - q1
    lower = q1 -1.5 * iqr
    upper = q3 +1.5 * iqr
    oulier = s.isnull().sum() !=0 # 0이면 이상치가 없다
    return oulier, np.round(s.mean(), 2), np.round(s.std(), 4)

In [11]:
species_list, feature_list, mean_list, std_list = [],[],[],[]
outlier_list, new_mean_list, new_std_list = [],[],[]
for species in iris.species.unique():
    for feature in iris.columns[:-1]:
        s = iris[iris.species == species][feature]
        species_list.append(species)
        feature_list.append(feature)
        mean_list.append(np.round(s.mean(), 2))
        std_list.append(np.round(s.std(), 4))

        outlier, new_mean, new_std = get_new_stat(s)
        outlier_list.append(outlier)
        new_mean_list.append(new_mean)
        new_std_list.append(new_std)

In [12]:

df = pd.DataFrame({
    'species': species_list, 'feature': feature_list,
    'mean': mean_list, 'std': std_list,
    'outlier': outlier_list, 'new_mean': new_mean_list, 'new_std': new_std_list
})
df.set_index(['species','feature'], inplace=True)
df

mean     std  outlier  new_mean  new_std
species    feature                                               
setosa     sepal_length  5.01  0.3525    False      5.01   0.3525
           sepal_width   3.43  0.3791    False      3.43   0.3791
           petal_length  1.46  0.1737    False      1.46   0.1737
           petal_width   0.25  0.1054    False      0.25   0.1054
versicolor sepal_length  5.94  0.5162    False      5.94   0.5162
           sepal_width   2.77  0.3138    False      2.77   0.3138
           petal_length  4.26  0.4699    False      4.26   0.4699
           petal_width   1.33  0.1978    False      1.33   0.1978
virginica  sepal_length  6.59  0.6359    False      6.59   0.6359
           sepal_width   2.97  0.3225    False      2.97   0.3225
           petal_length  5.55  0.5519    False      5.55   0.5519
           petal_width   2.03  0.2747    False      2.03   0.2747

## 2. Titanic - sns.load_dataset('titanic')

### 2.a. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오.
category1 카테고리는 다음과 같이 정의됨 
1) 20살이 넘으면 성별을 그대로 사용한다. 
2) 20살 미만이면 성별에 관계없이 “child”라고 한다. 

In [13]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [14]:
titanic['category1'] = titanic.apply(lambda x : x.sex if x.age > 20 else 'child',axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


### b. 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값이 되도록 titanic 데이터프레임을 고치시오.


In [15]:
titanic.age.mean()
titanic['age']=np.round(titanic.age.fillna(titanic.age.mean()),1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [16]:
titanic['category1'] = titanic.apply(lambda x : x.sex if x.age > 20 else 'child',axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


### c.성성별 , 선실 ( 별 , 출발지 embark_town 별 생존율을 구하시오

In [17]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


In [18]:
titanic.pivot_table('survived', 'class' )

,survived
class,
First,0.629630
Second,0.472826
Third,0.242363


In [19]:
titanic.pivot_table('survived', 'embark_town')

,survived
embark_town,
Cherbourg,0.553571
Queenstown,0.389610
Southampton,0.336957


In [20]:
titanic.pivot_table('survived', ['sex', 'class', 'embark_town']).head()

survived
sex    class  embark_town          
female First  Cherbourg    0.976744
              Queenstown   1.000000
              Southampton  0.958333
       Second Cherbourg    1.000000
              Queenstown   1.000000

In [21]:
titanic.groupby(['sex', 'class', 'embark_town'])[['survived']].mean().head()

survived
sex    class  embark_town          
female First  Cherbourg    0.976744
              Queenstown   1.000000
              Southampton  0.958333
       Second Cherbourg    1.000000
              Queenstown   1.000000

### 4.d.
타이타닉호 승객을 ‘ 미성년자’, 청년’청년’, 중년’중년’, 장년’장년’, 노년’ 나이 
그룹으로 나누고각 그룹별 생존율을 구하시오

In [22]:
bins = [1, 20, 30, 50, 70, 100]
labels = ['미성년자', '청년', '중년', '장년', '노년']
titanic['age_cat'] = pd.cut(titanic.age, bins, labels=labels)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_cat
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,청년
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,미성년자
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female,청년
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,청년
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,중년


In [23]:
titanic.pivot_table('survived', 'age_cat')

,survived
age_cat,
미성년자,0.424242
청년,0.334152
중년,0.423237
장년,0.355932
노년,0.200000


In [24]:
titanic.pivot_table('survived', ['age_cat', 'sex'])

survived
age_cat sex             
미성년자    female  0.671233
        male    0.228261
청년      female  0.723881
        male    0.142857
중년      female  0.779070
        male    0.225806
장년      female  0.941176
        male    0.119048
노년      male    0.200000

### e.
qcut 명령으로 세 개의 나이 그룹을 만들고 , 나이 그룹별 남녀 성비와 생존율을 구하시오

In [25]:
titanic['age_group'] = pd.qcut(titanic.age, 3, labels=['A1', 'A2', 'A3'])
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_cat,age_group
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,청년,A2
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,미성년자,A1
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female,청년,A2
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,청년,A2
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,중년,A3


In [26]:
titanic.groupby('age_group')[['survived']].mean()

,survived
age_group,
A1,0.411960
A2,0.335526
A3,0.405594


In [27]:
#성비 구하기 위한 gender : 남성이면 1 여성이면 0
titanic['gender'] = titanic.apply(lambda r: 1 if r.sex== 'male' else 0, axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1,age_cat,age_group,gender
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male,청년,A2,1
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child,미성년자,A1,0
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female,청년,A2,0
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male,청년,A2,1
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male,중년,A3,1


In [28]:
titanic.pivot_table('gender','age_group')

,gender
age_group,
A1,0.594684
A2,0.680921
A3,0.667832


In [29]:
titanic.pivot_table(['gender', 'survived'], 'age_group' )

,gender,survived
age_group,,
A1,0.594684,0.411960
A2,0.680921,0.335526
A3,0.667832,0.405594


## Mile Per Gallon

a. 배기량 대비 마력

In [30]:
mpg = sns.load_dataset('mpg')

In [33]:
mpg['hp_per_cc'] = mpg.horsepower / mpg.displacement
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,0.423453
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,0.471429
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,0.471698
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,0.493421
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,0.463576


### b.
name 으로부터 manufacturer( 제조사 와 모델을 추출하여
새로운 열 manufacturer 와 model 을 추가하고 , name 열은 삭제하시오

In [41]:
mpg['제조사'] = mpg.name.apply(lambda x : x.split()[0])
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc,제조사
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,0.423453,chevrolet
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,0.471429,buick
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,0.471698,plymouth
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,0.493421,amc
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,0.463576,ford


In [42]:
mpg['model'] = mpg.name.apply(lambda x : ' '.join(x.split()[1: ]))
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc,제조사,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,0.463576,ford,torino


In [43]:
del mpg['name']
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,제조사,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


### c.
엔진의 실린더 ( 갯수별 연비 ( 의 평균을 구하시오

In [44]:
mpg.groupby('cylinders')[['mpg']].mean()

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [45]:
mpg[['mpg', 'cylinders']].groupby('cylinders').mean()

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [46]:
mpg.pivot_table('mpg', 'cylinders')

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


### d.
생산지 ( oring)별 배기량 대비 마력 hp_per_cc 의 평균을 구하시오

In [47]:
mpg.pivot_table('hp_per_cc', 'origin')

,hp_per_cc
origin,
europe,0.740005
japan,0.789485
usa,0.512063


### e.
모델이 5 개 이상인 제조사에 대하여
연비 ( 의 평균이 가장 좋은 제조사 Top 5 를 구하시오

In [48]:
s=mpg.제조사.value_counts()
big = s[s>=5].index.tolist()
print(big)

['ford', 'chevrolet', 'plymouth', 'amc', 'dodge', 'toyota', 'datsun', 'buick', 'pontiac', 'volkswagen', 'honda', 'mercury', 'mazda', 'oldsmobile', 'fiat', 'peugeot', 'audi', 'chrysler', 'vw', 'volvo', 'renault']


In [52]:
# in 연산자 대신 isin 메소드사용
mpg[mpg.제조사.isin(big)].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,제조사,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


In [56]:
bigmpg = mpg[mpg.제조사.isin(big)]
bigmpg.pivot_table('mpg', '제조사').sort_values(by="mpg", ascending=False).head()

,mpg
제조사,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000


In [57]:
mpg.groupby('제조사').filter(lambda x: x.제조사.count() >= 5).head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,제조사,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


In [58]:
newmpg = mpg.groupby('제조사').filter(lambda x: x.제조사.count() >= 5)
newmpg.pivot_table('mpg', '제조사').sort_values(by='mpg', ascending=False).head()

,mpg
제조사,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000
